In [67]:
from concurrent.futures import ThreadPoolExecutor
import json
import os
from openai import OpenAI
import logging
from time import perf_counter, sleep
import re
from threading import RLock
from dotenv import load_dotenv

load_dotenv()

log_format = '[ %(levelname)s] [%(asctime)s] [%(module)s] [%(lineno)s] [%(message)s]'
logging.basicConfig(level=logging.DEBUG, format=log_format)

file_handler = logging.FileHandler('logfile.log')
file_handler.setLevel(logging.DEBUG)

formatter = logging.Formatter(log_format)
file_handler.setFormatter(formatter)

log = logging.getLogger()
log.addHandler(file_handler)

lock = RLock()
converted_qno = set()
failed_qno = []
reponse_data = []



In [68]:

SYSTEM_PROMPT = """
**You are a UPSC prelims question expert specializing in converting English MCQs to Hindi.**

**Given a question in English (statement) and a hint, convert it to Hindi (statement) suitable for the UPSC prelims exam, maintaining technical accuracy and UPSC context.**

**Input (JSON):**

* `statement`: Question statement with options in English.
* `hint`: Hint for the question in English.
    
**Output (JSON):**

* `statement`: Converted question statement with options in Hindi.
* `hint`: Hint for the question in Hindi.

**Example::1**

* Example Input::1
    "statement": "Consider following statements regarding the representation of States in the Parliament: 1. Delimitation of Constituencies is undertaken on the basis of census exercise to ensure that every State is represented in proportion to its population in both the Houses of Parliament. 2. Delimitation Commission is a constitutional body, the notification of whose orders cannot be challenged in a Court. 3. Territorial constituencies in States, at present, are based on the data of 2001 census, as the Constitution (87thAmendment) Act, 2003 enabled the delimitation exercise on the basis of 2001Census figures. 4. As it stands today, Constitution of India prohibits any delimitation exercise till 2031. Which of the statements given above are not correct ? (A) 1, 2 and 4only (B) 2, 3 and 4only (C) 1, 3 and 4only (D) 1, 2, 3 and 4"
    "hint": "Delimitation constituencies are NOT applicable to representation of states in Council of States. Though it is correct to say the Order of delimitation commission, once notified, cannot be challenged in any Court, Delimitation commission is NOT a constitutional body but a statutory body. The Constitution has prohibited the revision of representation of States in the Lok Sabha till 2026, but not the delimitation of the Lok Sabha and Assembly constituencies..."

* Example Output::1
    "statement": "संसद में राज्यों के प्रतिनिधित्व का प्रस्तावना से संबंधित निम्नलिखित कथनों को विचार करें: 1. निर्वाचनी सीमाओं का निर्धारण जनगणना अभ्यास के आधार पर किया जाता है ताकि सुनिश्चित किया जा सके कि प्रत्येक राज्य को संसद के दोनों सदनों में उसकी जनसंख्या के अनुपात में प्रतिनिधित्व मिले। 2. निर्वाचन सीमा आयोग एक संवैधानिक निकाय है, जिसके आदेशों की अधिसूचना को किसी भी न्यायालय में चुनौती नहीं की जा सकती है। 3. राज्यों में क्षेत्रीय निर्वाचनी सीमाएँ, वर्तमान में, 2001 की जनगणना के आंकड़ों पर आधारित हैं, क्योंकि संविधान (87वां संशोधन) अधिनियम, 2003 ने 2001 की जनगणना के आंकड़ों पर आधारित निर्वाचन का अभ्यास संभव बनाया। 4. जैसा कि आज है, भारतीय संविधान किसी भी निर्वाचन अभ्यास का कोई अभ्यस्त नहीं करता है जब तक 2031 तक। उपर्युक्त कथनों में से कौन सही नहीं है? (A) 1, 2 और 4 (B) केवल 2, 3 और 4 (C) केवल 1, 3 और 4 केवल (D) 1, 2, 3 और 4"
    "hint": "निर्वाचन सीमाएँ संसद के सदन मे प्रतिनिधित्व के लिए लागू नहीं हैं। यह सही है कि एक बार घोषित किए गए निर्वाचन आयोग के आदेश को किसी भी न्यायालय में चुनौती नहीं की जा सकती है, लेकिन निर्वाचन आयोग संवैधानिक निकाय नहीं है बल्कि एक वैधानिक निकाय है। संविधान ने राज्यों के प्रतिनिधित्व की संशोधन की प्रतिनिधित्व को 2026 तक रोका है, लेकिन संविधान में स्पष्ट नहीं किया है कि संविधान और विधानसभा क्षेत्रों की नियमन (delimitation) नहीं कर सकते हैं।.."
    
**Example::2**

* Example Input::2
    "statement": "Which of the following statements is/are true about the Gram Sabha? 1. All people living in a village or a group of villages are members of the Gram Sabha. 2. All the plans for work of Gram Panchayat have to be approved by Gram Sabha. 3. For better implementation of some specific tasks, Gram Sabha form committees. 4. The elected Secretary of the Gram Sabha calls the meeting and keeps a record of the proceedings. (A) 2 and 3 (B) 1, 3 and 4 (C) 2, 3 and 4 (D) 1,2,3,4"
    "hint": "Only adult villagers who have the right to vote can be member of Gram Sabha. Persons below 18 years of age can't become members. Gram Sabha plays a supervisory and monitoring role over Gram Panchayat by approving it plan of work. Gram Sabha form committees like construction, animal husbandry, etc to carry out some specific tasks. The Gram Panchayat has a Secretary who is also the Secretary of the Gram Sabha. This person is not an elected person but is appointed by the government. The Secretary is responsible for calling the meeting of the Gram Sabha and Gram Panchayat and keeping a record of the proceedings."

* Example Output::2
    "statement": "निम्नलिखित में से कौन से कथन ग्राम सभा के बारे में सही हैं? 1. ग्राम सभा के सभी लोग एक गाँव या एक समूह के गाँवों के सदस्य हैं। 2. ग्राम पंचायत के काम की सभी योजनाएँ ग्राम सभा द्वारा मंजूर करनी हैं। 3. कुछ विशिष्ट कार्यों के बेहतर कार्यान्वयन के लिए, ग्राम सभा समितियाँ बनाती है। 4. ग्राम सभा का चुनावित सचिव सभा को बुलाता है और कार्यवाहियों का रिकॉर्ड रखता है। (A) 2 और 3 (B) 1, 3 और 4 (C) 2, 3 और 4 (D) 1,2,3,4",
    "hint": "केवल मतदान करने का अधिकार वाले वयस्क ग्रामीण ग्राम सभा के सदस्य बन सकते हैं। 18 वर्ष से कम उम्र के व्यक्ति सदस्य नहीं बन सकते। ग्राम सभा ग्राम पंचायत की काम की योजना को मंजूरी देकर एक पर्यवेक्षक और निगरानी भूमिका निभाती है। ग्राम सभा निर्माण, पशुपालन आदि जैसी समूचे कुछ विशिष्ट कार्यों को संभालने के लिए समितियाँ बनाती है। ग्राम पंचायत के पास एक सचिव होता है जो ग्राम सभा का भी सचिव होता है। यह व्यक्ति चुना नहीं जाता है, बल्कि सरकार द्वारा नियुक्त होता है। सचिव को ग्राम सभा और ग्राम पंचायत की मीटिंग बुलाने और प्रक्रियाओं का रिकॉर्ड रखने का जिम्मेदारी होती है।"

**IMPORTANT INFORMATION**
* DONOT change the base statement format. The number of options should remain the same in the converted statement.
* DONOT forget to translate choices from end of each input `statement` to reponse `statement`.
* DONOT translate numbers
"""

In [69]:
def clean_text(text, clean=True):
    if not clean:
        return re.sub(r'\s+', ' ', text.strip())
        # return text.strip()
    return re.sub(r'\s+', ' ', text.replace("\\n", " ").replace("", " ").strip())


In [70]:
def get_data():
    input_data = []
    input_data_dict = {}
    qno = []
    with open('data.json', 'r') as fp:
        json_data = json.load(fp)
    for key, item in json_data.items():
        statement = clean_text(item['statement'])
        hint = clean_text(item['hint'])
        qno = item["Qno"]
        temp = {'statement': statement, 'hint': hint, "Qno": qno}
        input_data.append(temp)
        input_data_dict[key] = temp
    # save_input_data(input_data_dict)
    return input_data


In [71]:
def convert_question(input_data, OPENAI_KEY):
    qno = input_data.pop("Qno")
    try:
        USER_PROMPT = str(input_data)
        data = OpenAI.generate_completion(SYSTEM_PROMPT, USER_PROMPT, OPENAI_KEY)
        if not data:
            raise Exception('Opena ai didnot respond')
        data["Qno"] = qno
        converted_qno.append(qno)
        reponse_data.append(data)
        # with lock:
            # save_data(reponse_data)

    except Exception as e:
        log.error(str(e))
        # with lock:
        failed_qno.append(qno)

In [72]:
def get_converted_questions():
    question_path = 'converted/questions_1.json'
    if os.path.exists(question_path):
        with open(question_path, 'r') as fp:
            file_data = json.load(fp)
        log.info(f'From questions.json: {len(file_data)}')
        
        # return [data['Qno'] for data in file_data]

    no_path = 'converted/no.json'
    if os.path.exists(no_path):
        with open(no_path, 'r') as fp:
            file_data = json.load(fp)
            return file_data

    return []

In [73]:
def save_data():
    question_path = 'converted/questions_1.json'
    try:
        if os.path.exists(question_path):
            with open(question_path, 'r') as fp:
                file_data = json.load(fp)

            with open(f'converted/questions_1_copy.json', 'w') as fp:
                json.dump(file_data, fp, ensure_ascii=False)

        log.info(f'Saving {len(reponse_data)} data')   
        with open(question_path, 'w') as fp:
            json.dump(reponse_data, fp, ensure_ascii=False)
        return True

    except Exception as e:
        log.error(str(e))
        return None

In [74]:
question_path = 'converted/questions_1.json'
if os.path.exists(question_path):
        with open(question_path, 'r') as fp:
            file_data = json.load(fp)
            reponse_data = file_data
            
input_data = get_data()
converted_qno = get_converted_questions()
OPENAI_KEY_1 = os.getenv('OPENAI_KEY_1')
OPENAI_KEY_2 = os.getenv('OPENAI_KEY_2')

log.info(converted_qno)
log.info(len(converted_qno))

[ INFO] [2024-04-30 06:58:15,051] [420080836] [6] [From questions.json: 946]


[ INFO] [2024-04-30 06:58:15,054] [81978856] [12] [[0, 1, 10, 100, 1000, 1001, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 101, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 102, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1002, 103, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 104, 1040, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 105, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 106, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 107, 1070, 1071, 1072, 1073, 1075, 1076, 1077, 1078, 1079, 108, 1080, 1082, 1083, 1084, 1085, 1087, 1088, 1089, 109, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 11, 110, 1100, 1101, 1102, 1103, 1104, 1105, 1107, 1108, 1109, 111, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 112, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 113, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 114, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1

In [75]:
for data in input_data:
    qno = data["Qno"]
    if qno in converted_qno:
        continue
    log.info(f'Converting question {qno}')
    try:
        convert_question(data, OPENAI_KEY=OPENAI_KEY_1)
        save_data()
        log.info(f'Converted question {qno}')
    except Exception as e:
        log.error(str(e))
        


[ INFO] [2024-04-30 06:58:19,028] [3670692085] [5] [Converting question 1086]
[ INFO] [2024-04-30 06:58:19,030] [openai] [48] [Sending request to openai api.]
[ DEBUG] [2024-04-30 06:58:19,033] [connectionpool] [1055] [Starting new HTTPS connection (1): api.openai.com:443]
[ DEBUG] [2024-04-30 06:59:15,043] [connectionpool] [549] [https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None]
[ INFO] [2024-04-30 06:59:15,045] [openai] [56] [Total tokens: 3750 | Completion Tokens: 315]
[ INFO] [2024-04-30 06:59:15,046] [openai] [94] [Sending request to openai api for fixing response]
[ ERROR] [2024-04-30 06:59:15,047] [openai] [100] [Error occured while fixing response: OpenAI.get_headers() missing 1 required positional argument: 'OPENAI_KEY']
[ ERROR] [2024-04-30 06:59:15,047] [openai] [74] [Error occured while generating remarks: OpenAI.get_headers() missing 1 required positional argument: 'OPENAI_KEY']
[ ERROR] [2024-04-30 06:59:15,049] [728153040] [15] [Opena ai didnot re

KeyboardInterrupt: 

In [76]:
print(len(converted_qno))
with open ('converted/no.json', 'w') as fp:
    json.dump(converted_qno, fp)

995


In [18]:
unique_elements = []
seen_qnos = set()
question_path = 'converted/questions_1.json'
if os.path.exists(question_path):
    with open(question_path, 'r') as fp:
            file_data = json.load(fp)
    
for item in file_data:
    if item['Qno'] not in seen_qnos:
        unique_elements.append(item)
        seen_qnos.add(item['Qno'])

In [19]:
print(len(file_data))
print(len(unique_elements))

383
383


In [12]:
with open(question_path, 'w') as fp:
        json.dump(unique_elements, fp, ensure_ascii=False)